# Aplicar funciones a elementos de `Series` o `DataFrame`s
Cualquier procesado, manipulación o análisis de conjuntos de datos implica el aplicar transformaciones, funciones o calcular indicadores elemento a elemento, fila a fila o columna a columna. 

In [3]:
# Preliminares
import pandas as pd
import numpy as np
from numpy.random import default_rng

Para ilustrar, generamos un `DataFrame` usando un generador de números aleatorios.

In [11]:
# Usamos el generador por defecto del módulo random de numpy
DEFAULT_SEED = 314159
rng = default_rng(DEFAULT_SEED)
df_random = pd.DataFrame(
    rng.standard_normal(size=(5, 2)),
    columns=list("xy") # alternatively = "x,y".split(",")
)




# 
df_random

,x,y
0,-0.734428,0.902422
1,-0.263277,0.843977
2,1.741125,0.129503
3,-0.925700,-1.788521
4,0.824510,-1.253349


## Funciones universales `ufuncs` de `numpy`
Puesto que un `DataFrame` también es un array de `numpy`, podemos aplicarle cualquier función universal de `numpy`. Las funciones universales de `numpy`(ufuncs) se pueden consultar en la referencia [Available ufuncs](https://numpy.org/doc/stable/reference/ufuncs.html#available-ufuncs)

In [7]:
# Calculamos la exponencial de x e y, elemento a elemento
np.exp(df_random)

,x,y
0,0.479780,2.465567
1,0.768529,2.325597
2,5.703755,1.138263
3,0.396254,0.167207
4,2.280763,0.285547


In [9]:
# Elevamos x a la potencia y, elemento a elemento:
np.power(df_random["x"], df_random["y"])
# NaN producido por intentar hacer la raiz negativa

0         NaN
1         NaN
2    1.074455
3         NaN
4    1.273608
dtype: float64

Compare time used in np.exp vs loops

In [14]:
import timeit

# Snippet de forma non-pythonic
command = """
import numpy as np
import pandas as pd
from numpy.random import default_rng

DEFAULT_SEED = 314159
rng = default_rng(DEFAULT_SEED)

x = np.array(rng.standard_normal(1000))
y = x # para que sea de la misma len

for i, value in enumerate(x):
    y[i] = np.exp(value)
"""

timeit.timeit(command, number=1000)


0.01965512099991429

In [15]:
# Snippet de forma non-pythonic
command = """
import numpy as np
import pandas as pd
from numpy.random import default_rng

DEFAULT_SEED = 314159
rng = default_rng(DEFAULT_SEED)

x = np.array(rng.standard_normal(1000))
np.exp(x)
"""

timeit.timeit(command, number=1000)

0.018241273000057845

## También se pueden aplicar métodos de `pandas` directamente a todo un `DataFrame`

In [16]:
# Para obtener la suma, columna por columna
df_random.sum() # axis = 1 realiza la operacion para todas las filas

x    0.642230
y   -1.165968
dtype: float64

In [17]:
# Para obtener la media, columna por columna
df_random.mean()

x    0.128446
y   -0.233194
dtype: float64

In [18]:
# Para obtener la desviación típica, columna por columna
df_random.std()

x    1.128545
y    1.228945
dtype: float64

También podríamos calcular estos indicadores, fila por fila, usando el parámetro `axis=1`.

In [19]:
df_random.sum(axis=1)

0    0.167994
1    0.580700
2    1.870628
3   -2.714221
4   -0.428839
dtype: float64

Podéis encontrar y explorar la gran cantidad de métodos disponibles para un `DataFrame`en la [documentación](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) de `pandas`.

## Aplicar funciones a elementos de `Series` o `DataFrame`s (II)
En el vídeo anterior vimos cómo aplicar funciones universales de `numpy` o los métodos de `pandas` a `DataFrame`s. Pero puede ser que queramos aplicar nuestras propias funciones. 

Al igual que en el vídeo anterior, para ilustrar, generamos un `DataFrame` usando un generador de números aleatorios.

In [22]:
# Usamos el generador por defecto del módulo random de numpy (mismo df)
df = df_random

Empezamos por crear una función

In [20]:
# hemos usado la construcción if else
def mi_func(x):
    return "Es positivo" if x >= 0 else "Es negativo"

Para aplicar nuestra función a cada elemento del `DataFrame` usamos `applymap`

In [23]:
df.applymap(mi_func)

/tmp/ipykernel_7081/4025082953.py:1: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(mi_func)


,x,y
0,Es negativo,Es positivo
1,Es negativo,Es positivo
2,Es positivo,Es positivo
3,Es negativo,Es negativo
4,Es positivo,Es negativo


Si quisieramos aplicar nuestra función a cada elemento de una `Serie` usaríamos `apply`:

In [26]:
df["x"].apply(mi_func)

0    Es negativo
1    Es negativo
2    Es positivo
3    Es negativo
4    Es positivo
Name: x, dtype: object

## Las funciones anónimas en Python

Es muy útil tener la posibilidad de definir sobre la marcha una función sin darle un nombre y usarla como argumento de un método. Se hace con `lambda`.

In [27]:
# Para obtener el doble de cada elemento
df.applymap(lambda x: x ** 2)

/tmp/ipykernel_7081/3859039923.py:2: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df.applymap(lambda x: x ** 2)


,x,y
0,0.539385,0.814365
1,0.069315,0.712297
2,3.031515,0.016771
3,0.856920,3.198808
4,0.679817,1.570883


## El método `apply` para un `DataFrame`

El método `apply` si se aplica a un `DataFrame` debe tener como argumento a una función que se aplica a toda una columna o toda una fila, no a un elemento individual.

In [31]:
(df >= 0)

,x,y
0,False,True
1,False,True
2,True,True
3,False,False
4,True,False


In [29]:
# Ejemplo con una función que se aplica a un vector: calculamos el número de valores positivos por columna
df.apply(
    lambda serie: serie.apply(lambda y: y ** 2)
)

df.apply(
    lambda x: (x >= 0).sum() # (x >= 0) devuelve un vector de "true" o "false" (true = 1, false = 0)
)

x    2
y    3
dtype: int64

El método `apply` admite también el argumento `axis=1`, para que se aplique la función fila por fila.

In [30]:
df.apply(
    lambda x: (x >= 0).sum(),
    axis=1
)

0    1
1    1
2    2
3    0
4    1
dtype: int64

> Es muy recomendable evitar `apply` si es posible, y usar las funciones de `numpy` o los métodos de `pandas` que son muy optimizados y llevan a cabo la iteración en C.
`apply` lleva a cabo la iteración en Python.

En el caso anterior, podríamos haber usado directamente el método `sum` en `pandas`.

In [33]:
(df >= 0).sum()

x    2
y    3
dtype: int64

In [34]:
# Fila por fila:
(df >= 0).sum(axis=1)

0    1
1    1
2    2
3    0
4    1
dtype: int64